In [1]:
import pandas as pd
from pandas.core.arrays.period import timedelta

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import os

In [2]:
def missing_val_count_by_column (df) :
    nan_count = (df.isnull().sum())
    print(nan_count[nan_count > 0]) 
    
def check_nans (df):
    is_NaN = df. isnull()
    row_has_NaN = is_NaN. any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    return rows_with_NaN

In [4]:
train = pd.read_excel('./data/train_data/train.xlsx')
val = pd.read_excel('./data/val_data/validate.xlsx')
# train = pd.read_excel('/content/train.xlsx')
# val = pd.read_excel('/content/validate.xlsx')

In [5]:
print(train.shape, val.shape)
missing_val_count_by_column(train)
missing_val_count_by_column(val)

(1096, 25) (730, 25)
Series([], dtype: int64)
Series([], dtype: int64)


In [6]:
def create_avg_day(df):
    if 'avg_day' not in df.columns:
        df['avg_day']=df.iloc[:, 1:].mean(axis = 1)
    else:
        print("There's already a column avg_day! No need to create another one.")
        
        
def rename_prices(df):
    if 'PRICES' in df.columns:
        df.rename(columns={"PRICES": "datetime"}, inplace = True)
    else:
        print("There's no column PRICES.")    

In [7]:
create_avg_day(train)
rename_prices(train)

create_avg_day(val)
rename_prices(val)

Date formatter: https://github.com/d3/d3-time-format/tree/v2.2.3#locale_format

# Data Formatting

In [8]:
def dataformatting(df):
    #wide to long
    df = df.melt(id_vars=['datetime'], value_vars=df.columns[1:25]).sort_values(['datetime', 'variable'])
    df.reset_index(inplace=True, drop=True)
    
    #creating master time column, ulgy but works
    time = df['datetime'].copy()
    for d in range(len(df['datetime'])):
        time[d] = df['datetime'][d]+timedelta(hours = d%24) #decided not to go for the +1, so hour 1 is midnight, makes more sense, now it ends in 2009, otherwise the last measurement was 01.01.2010 00:00:00
    df['time'] = time
    
    #hour from string to int
    df['variable'] = df['variable'].map(lambda x:int(x[-2:]))
    
    #renaming, shullfing columns (not important)
    df.rename(columns={"datetime": "date", "variable": "hour", "value":"price"}, inplace = True)

    df['month'] = pd.DatetimeIndex(df['time']).month
    df['day'] = pd.DatetimeIndex(df['time']).day
    df['hour'] = pd.DatetimeIndex(df['time']).hour + 1

    cols = df.columns.tolist()
    cols = [cols[0]] + [cols[3]] + [cols[2]] + [cols[4]] + [cols[5]] + [cols[1]]
    df = df[cols]
    
    return df
    

TO DO:
- make pipeline for big and small dfs
- remove 2 first columns at the end!

# Features

Long Term: 
- (hourly) bollinger bands (5d, 14d, 30d)
- (hourly) EMA (5d, 14d, 30d)
- (daily) ATR (5d, 14d, 30d)
- (hourly) stochastic oscilator (5d + smoothed, 14d + smoothed, 30d + smoothed)

Short Term:
- (hourly) bollinger bands (8h, 24h)
- (hourly) EMA (8h, 24h)
- (hourly) stochastic oscilator (24h + smoothed)

## Bollinger Bands
A Bollinger Band is a technical analysis tool defined by a set of lines plotted two standard deviations (positively and negatively) away from a simple moving average (SMA) of the Stocks’ price Bollinger Bands allow traders to monitor and take advantage of shifts in price volatilities

Main Components of a Bollinger Bands:
 - Upper Band: The upper band is simply two standard deviations above the moving average of a stock’s price.
 - Middle Band: The middle band is simply the moving average of the stock’s price.
 - Lower Band: Two standard deviations below the moving average is the lower band.
 
Bollinger Bands allow traders to monitor and take advantage of shifts in price volatilities.

When calculating the SMA for Bollinger bands, traders typically use a 20 day SMA. Here is how you would calculate the SMA of a stock.

In [9]:
def bollinger_bands(df, hours):
    out = df.copy()
    rolled_mean = out['price'].rolling(hours).mean() 
    rolled_mean.reset_index(inplace=True, drop=True)
    for i in range (1,hours):
        short = out['price'][0:i]
        rolled_mean.loc[i-1] = short.mean()
    
    rolled_std = out['price'].rolling(hours).std() 
    rolled_std.reset_index(inplace=True, drop=True)
    for i in range (1,hours):
        # that's an estimation, first few points are quite bad but evens out quickly
        short = out['price'][0:i+5] #to make it less biased, i+5, so no 0 at the beginning etc
        rolled_std.loc[i-1] = short.std()

    bollinger_up = rolled_mean + rolled_std * 2 # Calculate top band
    bollinger_middle = rolled_mean
    bollinger_down = rolled_mean - rolled_std * 2 # Calculate bottom band

    out[f'{hours}h / {int(hours/24)}day bollinger down'] = bollinger_down
    out[f'{hours}h / {int(hours/24)}day bollinger middle'] = bollinger_middle
    out[f'{hours}h / {int(hours/24)}day bollinger up'] = bollinger_up

    return out

## Exponential moving average (EMA)
An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance on the most recent data points. The exponential moving average is also referred to as the exponentially weighted moving average. An exponentially weighted moving average reacts more significantly to recent price changes than a simple moving average (SMA), which applies an equal weight to all observations in the period.

One of the most popular EMA spans to use are: 8, 12, 20 (for short term trading) and 50, 200 (for long term)

Some traders use Fibonacci numbers (5, 8, 13, 21 ...) to select moving averages.

In general, the 50- and 200-day EMAs are used as indicators for long-term trends. When a stock price crosses its 200-day moving average, it is a technical signal that a reversal has occurred.

In [10]:
def get_ema(df, period_length:int):
    #returns a new df, which contains new column
    x = df.copy()
    x[f'{period_length}h / {int(period_length/24)}day EMA'] = x['price'].ewm(span=period_length).mean()
    return x

## Average true range (ATR)
The average true range (ATR) is a technical analysis indicator that measures market volatility by decomposing the entire range of an asset price for that period. ATR measures market volatility. It is typically derived from the 14-day moving average of a series of true range indicators.

In [11]:
def get_atr(df, period:int):
    x = df.copy()
    maxes = (x.groupby(['date']).max()['price'])
    mins = (x.groupby(['date']).min()['price'])
    closing_prices = df.loc[df['hour'] == 24]['price']

    maxes.reset_index(inplace=True, drop=True)
    mins.reset_index(inplace=True, drop=True)
    closing_prices.reset_index(inplace=True, drop=True)

    yesterday_prices = closing_prices.shift(1)
    yesterday_prices[0] = yesterday_prices[1] #taking care of NA
    
    tr1 = maxes-mins
    tr2 = abs(maxes - yesterday_prices)
    tr3 = abs(mins - yesterday_prices)

    tmp = pd.DataFrame({"A": tr1, "B": tr2, "C":tr3})
    true_range = tmp[["A", "B", "C"]].max(axis=1)
    a_tr = pd.Series([true_range[0]], dtype = np.float64)
    for i in range(1, len(true_range)):
        if i < period:
            a_tr[i] = (a_tr[i-1] * (i-1) + true_range[i]) / i
        else:
            a_tr[i] = (a_tr[i-1] * (period-1) + true_range[i]) / period

    #extending it so it's same for the day
    a_tr = a_tr.loc[a_tr.index.repeat(24)]
    a_tr.reset_index(inplace=True, drop=True)
    x[f'{period*24}h / {int(period)}day ATR'] = a_tr
    return x

# Stochastic Oscillator
ranges 0-100, measure of over/underbuying

In [12]:
def stochastic_osc(df, hours, smoothing):
  out = df.copy()
  maxes = out['price'].rolling(hours).max()
  mins = out['price'].rolling(hours).min()
  sliced = out['price'][0:hours-1]
  mins[0:hours-1] = min(sliced)
  maxes[0:hours-1] = max(sliced)
  oscillator = (out['price'] - mins) / (maxes - mins) * 100
  out[f'{hours}h / {int(hours/24)}stochastic oscillator'] = oscillator
  #usually you use both the oscillator and rolling average of it, this function adds both if bool flag smoothing, hardcoded 8 hours moving avg
  if smoothing:
    osc_smoothed = oscillator.rolling(8).mean()
    #simple NaN handling
    osc_smoothed[0:7] = osc_smoothed[7]
    out[f'{hours}h / {int(hours/24)}smoothed oscillator'] = osc_smoothed
  return out  

In [24]:
def scale_zero_one(df):
    x = df.copy()
    #returns df with all columns scaled to 0-1 individually
    #DANGER - DATA LEAKAGE
    for c in range (0, len(x.columns)):
        col = x.iloc[:,c].copy()
        norm_col=(col-col.min())/(col.max()-col.min())
        x.iloc[:,c] = norm_col
    return x


In [25]:
def build_big(data_f):
    df = data_f.copy()
    df = dataformatting(df)
    
    df = bollinger_bands(df, 8)
    df = bollinger_bands(df, 24)
    df = bollinger_bands(df, 120)
    df = bollinger_bands(df, 336)
    df = bollinger_bands(df, 720)
    
    df = get_ema(df, period_length=8)
    df = get_ema(df, period_length=24)
    df = get_ema(df, period_length=120)
    df = get_ema(df, period_length=336)
    df = get_ema(df, period_length=720)

    df = get_atr(df,5) #here operating on days!
    df = get_atr(df,14) #here operating on days!
    df = get_atr(df,30) #here operating on days!

    df = stochastic_osc(df, 24, True)
    df = stochastic_osc(df, 120, True)
    df = stochastic_osc(df, 336, True)
    df = stochastic_osc(df, 720, True)

    df.drop(columns = ['date', 'time'], inplace = True)

    df = scale_zero_one(df)
    
    return df

In [14]:
def build_small(data_f):
    df = data_f.copy()
    df = dataformatting(df)

    df = bollinger_bands(df, 24)

    df = get_ema(df, period_length=24)
    
    df = get_atr(df,5) #here operating on days!
    
    df = stochastic_osc(df, 24, False)
    
    df.drop(columns = ['date', 'time'], inplace = True)
    
    return df

In [17]:
categorical_train = build_small(train)
categorical_train.tail(5)

,price,month,day,hour,24h / 1day bollinger down,24h / 1day bollinger middle,24h / 1day bollinger up,24h / 1day EMA,120h / 5day ATR,24h / 1stochastic oscillator
26299,36.00,12,31,20,4.305981,27.137083,49.968186,31.306249,39.162491,83.177851
26300,29.00,12,31,21,4.268550,26.928333,49.588117,31.121749,39.162491,62.112549
26301,24.82,12,31,22,4.119311,26.788333,49.457356,30.617609,39.162491,49.533554
26302,31.28,12,31,23,4.118847,26.716667,49.314486,30.670600,39.162491,68.973819
26303,31.00,12,31,24,4.181444,26.842083,49.502723,30.696952,39.162491,68.131207


In [18]:
categorical_val = build_small(val)
categorical_val.head(5)

,price,month,day,hour,24h / 1day bollinger down,24h / 1day bollinger middle,24h / 1day bollinger up,24h / 1day EMA,120h / 5day ATR,24h / 1stochastic oscillator
0,13.20,1,1,1,8.144952,13.200000,18.255048,13.200000,32.49,37.550015
1,13.14,1,1,2,6.424676,13.170000,19.915324,13.168750,32.49,37.365343
2,11.83,1,1,3,3.987663,12.723333,21.459004,12.684818,32.49,33.333333
3,9.49,1,1,4,3.742843,11.915000,20.087157,11.783622,32.49,26.131117
4,8.63,1,1,5,3.544821,11.258000,18.971179,11.043593,32.49,23.484149


Creating DF for tabular learning

In [19]:
train_discrete_path = os.path.join(os.getcwd(),'data/train_data/train_discrete.npy')
val_discrete_path = os.path.join(os.getcwd(),'data/val_data/val_discrete.npy')

with open(train_discrete_path,'wb') as f: 
    np.save(f,categorical_train.to_numpy())

with open(val_discrete_path,'wb') as f:
    np.save(f,categorical_val.to_numpy())

In [26]:
train_preprocessed = build_big(train)
train_preprocessed.tail(5)


,price,month,day,hour,8h / 0day bollinger down,8h / 0day bollinger middle,8h / 0day bollinger up,24h / 1day bollinger down,24h / 1day bollinger middle,24h / 1day bollinger up,...,336h / 14day ATR,720h / 30day ATR,24h / 1stochastic oscillator,24h / 1smoothed oscillator,120h / 5stochastic oscillator,120h / 5smoothed oscillator,336h / 14stochastic oscillator,336h / 14smoothed oscillator,720h / 30stochastic oscillator,720h / 30smoothed oscillator
26299,0.014396,1.0,1.0,0.826087,0.887326,0.049522,0.021845,0.927897,0.068454,0.025909,...,0.054076,0.084765,0.831779,0.746648,0.640733,0.641253,0.476626,0.401412,0.251257,0.269925
26300,0.011596,1.0,1.0,0.869565,0.886183,0.047377,0.021189,0.927855,0.067776,0.025582,...,0.054076,0.084765,0.621125,0.719291,0.516112,0.618154,0.383923,0.400037,0.202388,0.258234
26301,0.009924,1.0,1.0,0.913043,0.884133,0.044859,0.020943,0.927689,0.067322,0.025470,...,0.054076,0.084765,0.495336,0.682412,0.441695,0.589704,0.328566,0.393917,0.173206,0.244510
26302,0.012508,1.0,1.0,0.956522,0.883956,0.044566,0.020870,0.927689,0.067090,0.025347,...,0.054076,0.084765,0.689738,0.690996,0.556703,0.590501,0.414117,0.407069,0.218305,0.242913
26303,0.012396,1.0,1.0,1.000000,0.884145,0.044767,0.020871,0.927759,0.067497,0.025509,...,0.054076,0.084765,0.681312,0.708625,0.551718,0.597544,0.410409,0.423663,0.216350,0.244009


In [31]:
train_preprocessed.day.unique()

array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
       0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ,
       0.33333333, 0.36666667, 0.4       , 0.43333333, 0.46666667,
       0.5       , 0.53333333, 0.56666667, 0.6       , 0.63333333,
       0.66666667, 0.7       , 0.73333333, 0.76666667, 0.8       ,
       0.83333333, 0.86666667, 0.9       , 0.93333333, 0.96666667,
       1.        ])

In [21]:
val_preprocessed = build_big(val)
val_preprocessed.head(5)

,price,month,day,hour,8h / 0day bollinger down,8h / 0day bollinger middle,8h / 0day bollinger up,24h / 1day bollinger down,24h / 1day bollinger middle,24h / 1day bollinger up,...,336h / 14day ATR,720h / 30day ATR,24h / 1stochastic oscillator,24h / 1smoothed oscillator,120h / 5stochastic oscillator,120h / 5smoothed oscillator,336h / 14stochastic oscillator,336h / 14smoothed oscillator,720h / 30stochastic oscillator,720h / 30smoothed oscillator
0,13.20,1,1,1,8.144952,13.200000,18.255048,8.144952,13.200000,18.255048,...,32.49,32.49,37.550015,23.264851,15.687283,9.719365,6.745078,4.371644,6.745078,4.371644
1,13.14,1,1,2,6.424676,13.170000,19.915324,6.424676,13.170000,19.915324,...,32.49,32.49,37.365343,23.264851,15.610132,9.719365,6.714395,4.371644,6.714395,4.371644
2,11.83,1,1,3,3.987663,12.723333,21.459004,3.987663,12.723333,21.459004,...,32.49,32.49,33.333333,23.264851,13.925678,9.719365,6.044490,4.371644,6.044490,4.371644
3,9.49,1,1,4,3.742843,11.915000,20.087157,3.742843,11.915000,20.087157,...,32.49,32.49,26.131117,23.264851,10.916806,9.719365,4.847865,4.371644,4.847865,4.371644
4,8.63,1,1,5,3.544821,11.258000,18.971179,3.544821,11.258000,18.971179,...,32.49,32.49,23.484149,23.264851,9.810981,9.719365,4.408080,4.371644,4.408080,4.371644


In [32]:
#to save to .npy file
np_train = train_preprocessed[train_preprocessed.columns[3:]].to_numpy()
with open('./data/train_data/train_big.npy', 'wb') as f:
    np.save(f, np_train)
    
np_val = val_preprocessed[val_preprocessed.columns[3:]].to_numpy()
with open('./data/val_data/val_big.npy', 'wb') as f:
    np.save(f, np_val)

In [23]:
#to open .npy file
with open('./data/train_data/train.npy', 'rb') as f:
    np_train = np.load(f)